In [7]:
from collections import OrderedDict
import copy
import requests
import json
import copy
import random
import operator
json_new = '/Users/Rhett/Desktop/DL_Project/SQUAD_DIR/ensemble_method1.json'
prediction_base = '/Users/Rhett/Desktop/DL_Project/SQUAD_DIR/squad_result_batch32_0%noise_predictions.json'
general_1 = '/Users/Rhett/Desktop/DL_Project/SQUAD_DIR/squad2.0_base_nbest_predictions-jun1.json'
general_2 = '/Users/Rhett/Desktop/DL_Project/SQUAD_DIR/squad_result_2.1-nbest_predictions_Rhett1.json'
general_3 = '/Users/Rhett/Desktop/DL_Project/SQUAD_DIR/squad2.0_base_nbest_predictions_jane1.json'
special_1 = '/Users/Rhett/Desktop/DL_Project/SQUAD_DIR/squad_result_2.3_batchsize32_30%noise_nbest_predictions.json'
special_2 = '/Users/Rhett/Desktop/DL_Project/SQUAD_DIR/squad2.0_base_result_15%noise_nbest_predictions.json'
special_3 = '/Users/Rhett/Desktop/DL_Project/SQUAD_DIR/squad_result_batch32_0%noise_nbest_predictions_jun2.json'
file = open(json_new, 'w')
with open(prediction_base) as s:
    with open(general_1) as f:
        with open(general_2) as g:
            with open(general_3) as h:
                with open(special_1) as x:
                    with open(special_2) as y:
                        with open(special_3) as z:
                            g1 = json.load(f)
                            g2 = json.load(g)
                            g3 = json.load(h)
                            s1 = json.load(x)
                            s2 = json.load(y)
                            s3 = json.load(z)
                            pred = json.load(s)

In [8]:
#无答案的给general model决策
# g是general model, s是specialized model。
g_list = [g1, g2, g3]
s_list = [s1, s2, s3]
#s1~30%, s2~15%, s3~0%
no_answer_idList = []
n = len(g_list)

for ID in list(pred.keys()):
    count_noAnswer = 0
    count_Answer = 0
    for g in g_list:
        if g[ID][0]['text'] == '':
            count_noAnswer += 1
        else: 
            count_Answer += 1
#     print ('no answer: ', count_noAnswer)
#     print ('answer: ', count_Answer)
    if count_noAnswer > count_Answer:
        pred[ID] = ''
        no_answer_idList.append(ID)

In [4]:
#方法1:有答案的，按照special model概率相加投票
for ID in list(g1.keys()):
    if ID in no_answer_idList: 
        continue
    else:
        v1 = OrderedDict()
        v2 = OrderedDict()
        v3 = OrderedDict()
#         print (ID)
    #     for v_unit in v_list:
        for rank in range(5):
            v1[s1[ID][rank]['text']] = s1[ID][rank]['probability']
            v2[s2[ID][rank]['text']] = s2[ID][rank]['probability']
            v3[s3[ID][rank]['text']] = s3[ID][rank]['probability']
        v_list = [v1, v2, v3]
        V = v1.copy()
        V.update(v2)
        V.update(v3)
        for key in V.keys():
            V[key] = 0
        for key in V.keys():
            for v0 in v_list:
                if key in v0:
                    V[key] = float(V[key]) + float(v0[key])
        sorted(V.items(),key = operator.itemgetter(1))
        pred[ID] = list(V.keys())[0]

In [5]:
#方法二，有答案的，直接按照special model排名最高的答案投票
for ID in list(g1.keys()):
    if ID in no_answer_idList: 
        continue
    else:  
        v1 = OrderedDict()
        v2 = OrderedDict()
        v3 = OrderedDict()
#         print (ID)
    #     for v_unit in v_list:
#         for rank in range(10):
        rank = 0
        v1[s1[ID][rank]['text']] = s1[ID][rank]['probability']
        v2[s2[ID][rank]['text']] = s2[ID][rank]['probability']
        v3[s3[ID][rank]['text']] = s3[ID][rank]['probability']
        v_list = [v1, v2, v3]
        V = v1.copy()
        V.update(v2)
        V.update(v3)
        for key in V.keys():
            V[key] = 0
        for key in V.keys():
            for v0 in v_list:
                if key in v0:
                    V[key] = float(V[key]) + 1
        sorted(V.items(),key = operator.itemgetter(1))
        pred[ID] = list(V.keys())[0]

In [9]:
#保存
pred = json.dumps(pred)
file.write(pred)
file.close()

In [10]:
g1

{'56ddde6b9a695914005b9628': [{'text': 'France',
   'probability': 0.9992731369473385,
   'start_logit': 7.960629940032959,
   'end_logit': 8.275537490844727},
  {'text': 'France.',
   'probability': 0.0006111587782031674,
   'start_logit': 7.960629940032959,
   'end_logit': 0.876110851764679},
  {'text': 'in France',
   'probability': 4.0225649271417367e-05,
   'start_logit': -2.1596486568450928,
   'end_logit': 8.275537490844727},
  {'text': 'a region in France',
   'probability': 3.715524737839166e-05,
   'start_logit': -2.239048480987549,
   'end_logit': 8.275537490844727},
  {'text': 'Normandy, a region in France',
   'probability': 3.113477933716607e-05,
   'start_logit': -2.415827989578247,
   'end_logit': 8.275537490844727},
  {'text': 'region in France',
   'probability': 6.236921004956884e-06,
   'start_logit': -4.023666858673096,
   'end_logit': 8.275537490844727},
  {'text': ', a region in France',
   'probability': 8.762803075852249e-07,
   'start_logit': -5.98622274398803